In [ ]:
pip install requests beautifulsoup4 pandas json


In [2]:
import requests
import time
import random
import pandas as pd
import datetime
import re
import os

# =============================================================================
# Hàm parse_vn_time_str
# =============================================================================
def parse_vn_time_str(time_str):
    """
    Chuyển đổi chuỗi thời gian tiếng Việt (ví dụ: "3 ngày trước", "5 giờ trước",
    "hôm qua", "30 giây trước") thành chuỗi định dạng "dd/mm/yy".
    
    Nếu chuỗi là "hôm qua", trừ 1 ngày khỏi thời điểm hiện tại.
    Nếu chuỗi có số và đơn vị (ngày, giờ, phút, giây) theo mẫu "x ... trước",
    trừ tương ứng khỏi thời điểm hiện tại.
    Nếu không parse được, mặc định trả về thời điểm hiện tại.
    
    Parameters:
        time_str (str): Chuỗi thời gian tiếng Việt.
        
    Returns:
        str: Chuỗi thời gian định dạng "dd/mm/yy".
    """
    now = datetime.datetime.now()         # Lấy thời điểm hiện tại
    if not time_str:                        # Nếu chuỗi rỗng hoặc None
        return None
    time_str = time_str.strip().lower()     # Loại bỏ khoảng trắng thừa và chuyển về chữ thường

    # Nếu chuỗi chứa "hôm qua", trừ đi 1 ngày
    if "hôm qua" in time_str:
        dt = now - datetime.timedelta(days=1)
    else:
        # Dùng biểu thức chính quy để tìm số và đơn vị (ngày, giờ, phút, giây)
        match = re.match(r"(\d+)\s+(ngày|giờ|phút|giây)\s+trước", time_str)
        if match:
            val = int(match.group(1))      # Lấy số lượng (ví dụ: 3, 5, 30)
            unit = match.group(2)          # Lấy đơn vị ("ngày", "giờ", "phút", "giây")
            if unit == "ngày":
                dt = now - datetime.timedelta(days=val)
            elif unit == "giờ":
                dt = now - datetime.timedelta(hours=val)
            elif unit == "phút":
                dt = now - datetime.timedelta(minutes=val)
            elif unit == "giây":
                dt = now - datetime.timedelta(seconds=val)
        else:
            # Nếu không khớp biểu thức, mặc định sử dụng thời điểm hiện tại
            dt = now

    # Trả về chuỗi định dạng "dd/mm/yy"
    return dt.strftime("%d/%m/%y")


# =============================================================================
# CẤU HÌNH REQUEST
# =============================================================================

# Cookie (nếu cần) để vượt qua hạn chế của server
cookies = {
    '_cfuvid': '8Ts694nmmK71_7TANcDMtfYe0q6_TtIxmRoeEi2V.pM-1739949090257-0.0.1.1-604800000'
}

# Headers giả lập trình duyệt
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'vi,en;q=0.9,en-GB;q=0.8,en-US;q=0.7',
    'Referer': 'https://www.nhatot.com/',
    'Connection': 'keep-alive',
    'Origin': 'https://www.nhatot.com',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'cross-site'
}

# Tham số truy vấn cho API của Chợ Tốt
params = {
    'limit': '10',                     # Số tin đăng mỗi trang
    'protection_entitlement': 'true',
    'page': '2',                        # Số trang (sẽ được cập nhật trong vòng lặp)
    'cg': '1000',                      # Mã danh mục (ví dụ: Nhà đất)
    'region_v2': '12000',              # Khu vực (ví dụ: Hà Nội)
    'st': 's,k',                       # Trạng thái sản phẩm
    'key_param_included': 'true'
}


# =============================================================================
# CẤU HÌNH CRAWL & LƯU FILE
# =============================================================================

MAX_PAGES = 10               # Số trang tối đa cần crawl
CHUNK_SIZE = 50                # Số trang mỗi khối để ghi ra CSV
OUTPUT_FILE = 'house_data.csv'   # Tên file CSV đầu ra

# Nếu file CSV đã tồn tại, xoá nó để lưu dữ liệu mới
if os.path.exists(OUTPUT_FILE):
    os.remove(OUTPUT_FILE)

# Global counter dùng để đếm số dòng đã ghi (dùng làm STT)
global_row_counter = 0

# Danh sách tạm chứa dữ liệu crawl được (dạng list of dict)
house_data_chunk = []


# =============================================================================
# HÀM LƯU DỮ LIỆU RA FILE CSV
# =============================================================================
def save_chunk_to_csv(data_list, filename):
    """
    Lưu danh sách dữ liệu (list of dict) ra file CSV theo chế độ append.
    
    Các bước thực hiện:
      1. Tạo DataFrame từ danh sách dữ liệu.
      2. Loại bỏ các bản ghi trùng lặp dựa trên cột 'ID'.
      3. Ép kiểu số cho các cột quan trọng (nếu chuyển không được, gán NaN).
      4. Chuyển đổi cột thời gian sang kiểu datetime theo định dạng "dd/mm/yy".
      5. Gán index liên tục (STT) dựa trên biến global_row_counter.
      6. Lưu DataFrame ra file CSV (append mode), sử dụng index với nhãn "STT".
    
    Parameters:
        data_list (list): Danh sách dữ liệu crawl được.
        filename (str): Tên file CSV để lưu.
    """
    if not data_list:
        return

    global global_row_counter  # Sử dụng biến toàn cục để cập nhật STT

    # Tạo DataFrame từ danh sách dữ liệu
    df_chunk = pd.DataFrame(data_list)

    # Loại bỏ các bản ghi trùng lặp dựa trên cột 'ID'
    df_chunk.drop_duplicates(subset=['ID', 'Thoi_Gian'], keep='first', inplace=True)


    # Ép kiểu số cho các cột: nếu chuyển đổi không được, gán giá trị NaN
    df_chunk['Dien_Tich'] = pd.to_numeric(df_chunk['Dien_Tich'], errors='coerce')
    df_chunk['Gia'] = pd.to_numeric(df_chunk['Gia'], errors='coerce')
    df_chunk['Gia/m2'] = pd.to_numeric(df_chunk['Gia/m2'], errors='coerce')
    df_chunk['So_Tang'] = pd.to_numeric(df_chunk['So_Tang'], errors='coerce')
    df_chunk['So_Phong_Ngu'] = pd.to_numeric(df_chunk['So_Phong_Ngu'], errors='coerce')
    df_chunk['So_Nha_Ve_Sinh'] = pd.to_numeric(df_chunk['So_Nha_Ve_Sinh'], errors='coerce')

    # Chuyển đổi cột Thoi_Gian sang kiểu datetime theo định dạng "dd/mm/yy"
    df_chunk['Thoi_Gian'] = pd.to_datetime(df_chunk['Thoi_Gian'], format='%d/%m/%y', errors='coerce')

    # Gán lại index cho DataFrame sao cho STT nối tiếp từ global_row_counter
    df_chunk.index = range(global_row_counter, global_row_counter + len(df_chunk))
    global_row_counter += len(df_chunk)

    # Ghi DataFrame ra file CSV ở chế độ append:
    # Nếu file chưa tồn tại thì ghi header, còn nếu đã tồn tại thì không ghi header.
    header_needed = not os.path.exists(filename)
    df_chunk.to_csv(filename, mode='a', index=True, index_label='STT', header=header_needed)


# =============================================================================
# QUÁ TRÌNH CRAWL DỮ LIỆU
# =============================================================================
for i in range(3, MAX_PAGES + 1):
    # Cập nhật số trang cho tham số truy vấn
    params['page'] = i

    try:
        # Gửi request GET đến API của Chợ Tốt
        response = requests.get(
            'https://gateway.chotot.com/v1/public/ad-listing',
            #headers=headers,
            params=params,
            #cookies=cookies,
            timeout=30
        )
    except requests.exceptions.RequestException as e:
        print(f'[ERROR] Trang {i} gặp lỗi: {e}')
        continue  # Nếu lỗi, bỏ qua trang hiện tại

    if response.status_code == 200:
        print(f'[INFO] Trang {i} request thành công.')
        # Lấy danh sách tin đăng (nếu không có, trả về list rỗng)
        ads_list = response.json().get('ads', [])
        
        # Nếu không còn tin đăng, dừng crawl sớm (tùy chọn)
        if not ads_list:
            print(f'[INFO] Không còn tin đăng ở trang {i}, dừng crawl.')
            break

        # Xử lý từng tin đăng trong danh sách
        for record in ads_list:
            # Lấy trường "date" và chuyển đổi sang định dạng "dd/mm/yy"
            raw_date = record.get('date')
            converted_date = parse_vn_time_str(raw_date) if raw_date else None

            # Thêm thông tin tin đăng vào danh sách tạm house_data_chunk
            house_data_chunk.append({
                'ID': record.get('ad_id'),
                'Dien_Tich': record.get('living_size'),
                'Gia': record.get('price'),
                'Gia/m2': record.get('price_million_per_m2'),
                'Ten_Duong': record.get('street_name'),
                'Ten_Phuong': record.get('ward_name'),
                'Ten_Quan': record.get('area_name'),
                'Ten_Tinh': record.get('region_name'),
                'Loai_Hinh': record.get('category_name'),
                'So_Tang': record.get('floors'),
                'So_Phong_Ngu': record.get('rooms'),
                'So_Nha_Ve_Sinh': record.get('toilets'),
                'Thoi_Gian': converted_date,   # Thời gian đã chuyển đổi
                'Phap_Ly': record.get('property_legal_document')
            })
    else:
        print(f'[WARN] Trang {i} request thất bại, status code: {response.status_code}')
    
    # Nghỉ ngẫu nhiên từ 1 đến 3 giây để tránh gửi request quá nhanh
    #time.sleep(random.uniform(1, 3))
    
    # Sau mỗi CHUNK_SIZE trang, lưu dữ liệu tạm ra file CSV và xoá danh sách tạm
    if i % CHUNK_SIZE == 0:
        save_chunk_to_csv(house_data_chunk, OUTPUT_FILE)
        house_data_chunk.clear()
        # Có thể thêm thời gian nghỉ dài hơn sau mỗi khối nếu cần (ví dụ: time.sleep(60))

# Nếu còn dữ liệu chưa được lưu, lưu nốt vào file CSV
if house_data_chunk:
    save_chunk_to_csv(house_data_chunk, OUTPUT_FILE)
    house_data_chunk.clear()

# =============================================================================
# XÓA những hàng có ít nhất 1 cột trống (NaN)
# =============================================================================
# Đọc lại file CSV để đảm bảo tất cả dữ liệu đã được xử lý
df = pd.read_csv(OUTPUT_FILE)
# Xóa các hàng có bất kỳ giá trị nào NaN
df.dropna(axis=0, how='any', inplace=True)

# Lưu lại file CSV sau khi xóa những hàng có cột trống
df.to_csv(OUTPUT_FILE, index=False)

# =============================================================================
# HIỂN THỊ TỔNG SỐ DÒNG CRAWL ĐƯỢC
# =============================================================================
print(f'[INFO] Crawl hoàn tất. Tổng số dòng crawl được: {global_row_counter}')
print(f'[INFO] File CSV lưu tại: {OUTPUT_FILE}')


[INFO] Trang 3 request thành công.
[INFO] Trang 4 request thành công.
[INFO] Trang 5 request thành công.
[INFO] Trang 6 request thành công.
[INFO] Trang 7 request thành công.
[INFO] Trang 8 request thành công.
[INFO] Trang 9 request thành công.
[INFO] Trang 10 request thành công.
[INFO] Crawl hoàn tất. Tổng số dòng crawl được: 10
[INFO] File CSV lưu tại: house_data.csv
